<a href="https://colab.research.google.com/github/dauvannam321/FineTuneVinaLLaMA/blob/main/FineTuneVinaLLaMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Install libraries**

In [ ]:
! pip install -q -U bitsandbytes
! pip install -q -U datasets
! pip install -q -U git+https://github.com/huggingface/transformers.git
! pip install -q -U git+https://github.com/huggingface/peft.git
! pip install -q -U git+https://github.com/huggingface/accelerate.git
! pip install -q -U loralib
! pip install -q -U einops
! pip install -q -U googletrans==3.1.0a0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 6.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (

# **Import libraries**

In [ ]:
import json
import os
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers

from googletrans import Translator
from pprint import pprint
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# **Init Model**

In [ ]:
MODEL_NAME = "vilm/vinallama-7b-chat"

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map = "auto",
    trust_remote_code = True,
    quantization_config = bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r = 16,
    lora_alpha = 32,
    target_modules = [
        "q_proj",
        "up_proj",
        "o_proj",
        "k_proj",
        "down_proj",
        "gate_proj",
        "v_proj"
    ],
    lora_dropout = 0.05,
    bias = "none",
    task_type = "CAUSAL_LM"
)

model = get_peft_model(model, config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.91G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.80G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.67M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
prompt = """
<|im_start|>system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
<|im_end|>
<|im_start|>user
Viết một hàm tính tổng hai số trong python
<|im_end|>
<|im_start|>assistant
""".strip()

In [ ]:
%%time
device = 'cuda' if torch.cuda.is_available () else 'cpu'

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
    input_ids = encoding.input_ids,
    attention_mask = encoding.attention_mask,
    generation_config = generation_config
  )

print(tokenizer.decode(outputs[0] ,skip_special_tokens = True))

<|im_start|> system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
 
<|im_start|> user
Viết một hàm tính tổng hai số trong python
 
<|im_start|> assistant
Dưới đây là một hàm Python nhận hai số làm đầu vào và trả về tổng của chúng:

```python
def add_two_numbers(a, b):
    return a + b
```

Bạn có thể sử dụng hàm này bằng cách gọi nó với hai số làm đối số. Ví dụ, nếu bạn muốn tìm tổng của 5 và 3, bạn có thể làm như sau:

```python
result = add_two_numbers(5, 3)
print(result)  # Output: 8
```

Hàm `add_two_numbers` nhận hai tham số, `a` và `b`, đại diện cho hai số bạn muốn cộng. Sau đó, nó sử dụng toán tử `+` để cộng hai số này lại với nhau và trả về kết quả.
CPU times: user 29.3 s, sys: 599 ms, total: 29.9 s
Wall time: 39.2 s


# **Load data**

In [ ]:
data = load_dataset('alespalla/chatbot_instruction_prompts')

Generating train split:   0%|          | 0/258042 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/64511 [00:00<?, ? examples/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['response', 'prompt'],
        num_rows: 258042
    })
    test: Dataset({
        features: ['response', 'prompt'],
        num_rows: 64511
    })
})

# **Convert data to right format**

In [ ]:
def generate_prompt(data_point):
  translator = Translator()
  vn_prompt = translator.translate(data_point['prompt'], src='en', dest='vi').text
  vn_response = translator.translate(data_point['response'], src='en', dest='vi').text
  return f"""
<|im_start|>system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
<|im_end|>
<|im_start|>user
{vn_prompt}
<|im_end|>
<|im_start|>assistant
{vn_response}
""".strip()

In [ ]:
def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)

  return tokenized_full_prompt

In [ ]:
data = data['train'].shard(num_shards=50, index=0).filter(lambda sample:sample['response'] != '' and sample['prompt'] != '').shuffle().map(generate_and_tokenize_prompt)


Filter:   0%|          | 0/5161 [00:00<?, ? examples/s]

Map:   0%|          | 0/5161 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


# **Training**

In [ ]:
training_args = transformers.TrainingArguments(
  per_device_train_batch_size = 1,
  gradient_accumulation_steps = 4,
  num_train_epochs = 1,
  learning_rate = 2e-4,
  fp16 = True,
  save_total_limit = 3,
  logging_steps = 1,
  output_dir = "/content/drive/MyDrive/Colab Notebooks/VinaLLaMA_For_Chatbot/experiments",
  optim = "paged_adamw_8bit",
  lr_scheduler_type = "cosine",
  warmup_ratio = 0.05,
)

trainer = transformers.Trainer(
  model = model,
  train_dataset = data,
  args = training_args,
  data_collator = transformers.DataCollatorForLanguageModeling(tokenizer , mlm = False)
)

In [ ]:
model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,3.772600
2,3.650700
3,3.692100
4,4.341900
5,3.750200
6,3.867000
7,3.132600
8,3.606200
9,3.195400
10,3.199500


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=1290, training_loss=1.2801281457261522, metrics={'train_runtime': 8913.4024, 'train_samples_per_second': 0.579, 'train_steps_per_second': 0.145, 'total_flos': 2.9498483519225856e+16, 'train_loss': 1.2801281457261522, 'epoch': 1.0})

# **Test**

In [ ]:
%%time
device = 'cuda' if torch.cuda.is_available () else 'cpu'

prompt = """
<|im_start|>system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
<|im_end|>
<|im_start|>user
Mô tả về thành phố Đà Lạt
<|im_end|>
<|im_start|>assistant
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
    input_ids = encoding.input_ids,
    attention_mask = encoding.attention_mask,
    generation_config = generation_config
  )

print(tokenizer.decode(outputs[0] ,skip_special_tokens = True))

<|im_start|> system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
 
<|im_start|> user
Mô tả về thành phố Đà Lạt
 
<|im_start|> assistant
Đà Lạt là một thành phố đẹp như tranh vẽ nằm ở vùng cao nguyên của Việt Nam. Đây là một trong những thành phố đẹp nhất Việt Nam với những ngọn đồi xanh tươi, những ngôi nhà cổ và những con đường quanh co. Thành phố này là một điểm đến phổ biến cho du khách, mang đến cho họ một khung cảnh yên bình và đẹp như tranh vẽ. Đà Lạt nổi tiếng với khí hậu mát mẻ, khiến nó trở thành nơi nghỉ dưỡng lý tưởng vào mùa hè. Thành phố cũng là nơi có nhiều nhà hàng, cửa hàng và các điểm tham quan, chẳng hạn như Bảo tàng Điêu khắc Điêu khắc và Vườn hoa Đà Lạt. Đà Lạt là một thành phố quyến rũ, mang lại sự pha trộn hoàn hảo giữa lịch sử, văn hóa và vẻ đẹp tự nhiên. Đó là một điểm đến phải ghé thăm đối với bất kỳ ai muốn trải nghiệm vẻ đẹp của Việt Nam.
CPU times: user 2min 8s, sys: 1min 2s, total: 3min 10s
Wall time: 3min 21s


In [ ]:
%%time
device = 'cuda' if torch.cuda.is_available () else 'cpu'

prompt = """
<|im_start|>system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
<|im_end|>
<|im_start|>user
Dịch "Chào buổi sáng" sang tiếng Pháp
<|im_end|>
<|im_start|>assistant
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
    input_ids = encoding.input_ids,
    attention_mask = encoding.attention_mask,
    generation_config = generation_config
  )

print(tokenizer.decode(outputs[0] ,skip_special_tokens = True))

<|im_start|> system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
 
<|im_start|> user
Dịch "Chào buổi sáng" sang tiếng Pháp
 
<|im_start|> assistant
Bonjour.

Bonjour là từ tiếng Pháp có nghĩa là "Chào buổi sáng". Nó được sử dụng để chào hỏi ai đó vào buổi sáng. Nó cũng có thể được sử dụng như một lời chào chung trong suốt cả ngày.
CPU times: user 30.1 s, sys: 13.6 s, total: 43.6 s
Wall time: 43.8 s
